In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from datasets import load_dataset

**加载`tokenizer`**

In [3]:
tokenizer = LlamaTokenizer.from_pretrained("yahma/llama-13b-hf")

**加载数据**

样例数据：
```json
{
    "instruction": "给出一座英国城市，写一个对主要旅游景点的描述",
    "input": "伦敦",
    "output": "伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年。 今天，游客可以探索宏伟的大厅、中世纪房间和白塔式建筑，并参观皇冠上的珠宝、传说中的大乌鸦，并查看博物馆里的各种各样的有趣物品。"
}
```

In [4]:
from datasets import Dataset
demo_data = [
    {
        "instruction": "给出一座英国城市，写一个对主要旅游景点的描述",
        "input": "伦敦",
        "output": "伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年。 今天，游客可以探索宏伟的大厅、中世纪房间和白塔式建筑，并参观皇冠上的珠宝、传说中的大乌鸦，并查看博物馆里的各种各样的有趣物品。"
    },
    {
        "instruction": "给出一座英国城市，写一个对主要旅游景点的描述",
        "input": "伦敦",
        "output": "伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年."
    }
]
data = Dataset.from_list(demo_data)

**打印数据**

In [5]:
data[0]

{'instruction': '给出一座英国城市，写一个对主要旅游景点的描述',
 'input': '伦敦',
 'output': '伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年。 今天，游客可以探索宏伟的大厅、中世纪房间和白塔式建筑，并参观皇冠上的珠宝、传说中的大乌鸦，并查看博物馆里的各种各样的有趣物品。'}

**定义Prompter**

`Prompter`是用来把原始数据镶嵌到一个模板里面，让最终输入到模型里面的数据更有规范。alpaca-lora中的Prompter代码如下：

In [6]:
import os.path as osp
import json
from typing import Union
class Prompter(object):
    # __slots__的作用：Prompter 只允许添加template和_verbose属性
    # _verbose是True的时候会打印一些日子信息；
    # 所以最核心的就是template
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

Prompter的作用是给定`instruction`, `input`和`output`，生成一个模版式的字符串，这个字符串会被送入到模型中去；

所以首先我们需要一个模版，来描述一下任务；

在Prompter中，默认的`template_name`是`alpaca`，所以对应的模版就是`templates/alpaca.json`中所写的内容，我们将其打印出来看看：

In [7]:
import json
template = json.load(open("templates/alpaca.json"))
print(json.dumps(template, indent=2, ensure_ascii=False))
print(template['prompt_input'])
print(template['prompt_no_input'])

{
  "description": "Template used by Alpaca-LoRA.",
  "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
  "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
  "response_split": "### Response:"
}
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:



所以模版中包含四个字段：

- description： 用来描述这个模版是啥模板的，不会在训练的时候加入到输入中去；
- prompt_input：如果原始数据的字段中有`input`这个字段，就是它对应的`prompt`
- prompt_no_input：如果原始数据的字段中没有`input`这个字段，就是它对应的`prompt`
- response_split：咋知道哪儿是输入的部分，哪儿是`response`，这个就是在模型训练好，用来生成文本的时候，根据这个字段的内容把模型的输出给切开，从而得到想要的`response`

首先，把`instruction`和`input`填充到这个模板里面，（如果没有`input`，使用不带input的prompt，填充`instruction`就可以了。）

其次，在训练的时候，会把原始输入的json里面的`output`字段的内容和上面填充的文本直接拼起来。返回即可。

用上面的例子的话，返回的结果长这样：

In [8]:
instruction, input, output = data[0]['instruction'], data[0]['input'], data[0]['output']
print(template['prompt_input'].format(instruction=instruction, input=input)+output)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
给出一座英国城市，写一个对主要旅游景点的描述

### Input:
伦敦

### Response:
伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年。 今天，游客可以探索宏伟的大厅、中世纪房间和白塔式建筑，并参观皇冠上的珠宝、传说中的大乌鸦，并查看博物馆里的各种各样的有趣物品。


**这个就是加入了模板之后，输入到模型中去的样子。**

下一步我们需要做的时候，是去做tokenize。

In [9]:
prompter = Prompter(template_name='alpaca')
train_on_inputs = True # 是否让 加上了instruction和input 的模板的部分参与训练，如果是False的话，就只有output的部分参与训练；
add_eos_token = True
cutoff_len = 512

In [10]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

In [11]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt

In [12]:
data = data.map(generate_and_tokenize_prompt)
for key, value in data[0].items():
    print(f"{key}: {value}")

instruction: 给出一座英国城市，写一个对主要旅游景点的描述
input: 伦敦
output: 伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年。 今天，游客可以探索宏伟的大厅、中世纪房间和白塔式建筑，并参观皇冠上的珠宝、传说中的大乌鸦，并查看博物馆里的各种各样的有趣物品。
input_ids: [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 31999, 30544, 30287, 31780, 31144, 30356, 30626, 30461, 30214, 31479, 30287, 30502, 30783, 30888, 30698, 233, 154, 136, 233, 187, 187, 31495, 30940, 30210, 233, 146, 146, 235, 194, 179, 13, 13, 2277, 29937, 10567, 29901, 13, 231, 191, 169, 233, 152, 169, 13, 13, 2277, 29937, 13291, 29901, 13, 231, 191, 169, 233, 152, 169, 30878, 235, 148, 154, 30548, 30210, 233, 154, 136, 233, 187, 187, 31495, 30940, 30577, 30287, 30392, 231, 191, 169, 233, 152, 169, 31831, 30267, 29871, 30810, 31780, 232, 145, 137, 30911, 233, 133, 163, 31347, 30210, 30626, 232, 163, 164, 30886, 30909, 29896, 29

In [13]:
train_on_inputs = False
data = data.map(generate_and_tokenize_prompt)
for key, value in data[0].items():
    print(f"{key}: {value}")

instruction: 给出一座英国城市，写一个对主要旅游景点的描述
input: 伦敦
output: 伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年。 今天，游客可以探索宏伟的大厅、中世纪房间和白塔式建筑，并参观皇冠上的珠宝、传说中的大乌鸦，并查看博物馆里的各种各样的有趣物品。
input_ids: [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 31999, 30544, 30287, 31780, 31144, 30356, 30626, 30461, 30214, 31479, 30287, 30502, 30783, 30888, 30698, 233, 154, 136, 233, 187, 187, 31495, 30940, 30210, 233, 146, 146, 235, 194, 179, 13, 13, 2277, 29937, 10567, 29901, 13, 231, 191, 169, 233, 152, 169, 13, 13, 2277, 29937, 13291, 29901, 13, 231, 191, 169, 233, 152, 169, 30878, 235, 148, 154, 30548, 30210, 233, 154, 136, 233, 187, 187, 31495, 30940, 30577, 30287, 30392, 231, 191, 169, 233, 152, 169, 31831, 30267, 29871, 30810, 31780, 232, 145, 137, 30911, 233, 133, 163, 31347, 30210, 30626, 232, 163, 164, 30886, 30909, 29896, 29

In [14]:
tokenizer.pad_token_id

In [15]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>']

In [75]:
tokenizer.all_special_ids

[1, 2, 0]

In [2]:
import transformers
"LlamaTokenizer" in transformers._import_structure["models.llama"]

/home/duhu/anaconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [17]:
data[0]

{'instruction': '给出一座英国城市，写一个对主要旅游景点的描述',
 'input': '伦敦',
 'output': '伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年。 今天，游客可以探索宏伟的大厅、中世纪房间和白塔式建筑，并参观皇冠上的珠宝、传说中的大乌鸦，并查看博物馆里的各种各样的有趣物品。',
 'input_ids': [1,
  13866,
  338,
  385,
  15278,
  393,
  16612,
  263,
  3414,
  29892,
  3300,
  2859,
  411,
  385,
  1881,
  393,
  8128,
  4340,
  3030,
  29889,
  14350,
  263,
  2933,
  393,
  7128,
  2486,
  1614,
  2167,
  278,
  2009,
  29889,
  13,
  13,
  2277,
  29937,
  2799,
  4080,
  29901,
  13,
  31999,
  30544,
  30287,
  31780,
  31144,
  30356,
  30626,
  30461,
  30214,
  31479,
  30287,
  30502,
  30783,
  30888,
  30698,
  233,
  154,
  136,
  233,
  187,
  187,
  31495,
  30940,
  30210,
  233,
  146,
  146,
  235,
  194,
  179,
  13,
  13,
  2277,
  29937,
  10567,
  29901,
  13,
  231,
  191,
  169,
  233,
  152,
  169,
  13,
  13,
  2277,
  29937,
  13291,
  29901,
  13,
  231,
  191,
  169,
  233,
  152,
  169,
  30878,
  235,
  148,
  154,
  30548,
  30210,
  233

In [18]:
data[1]

{'instruction': '给出一座英国城市，写一个对主要旅游景点的描述',
 'input': '伦敦',
 'output': '伦敦最著名的旅游景点之一是伦敦塔。 这座历史悠久的城堡建于1000多年前，曾经用作监狱、皇家住所和堡垒长达600多年.',
 'input_ids': [1,
  13866,
  338,
  385,
  15278,
  393,
  16612,
  263,
  3414,
  29892,
  3300,
  2859,
  411,
  385,
  1881,
  393,
  8128,
  4340,
  3030,
  29889,
  14350,
  263,
  2933,
  393,
  7128,
  2486,
  1614,
  2167,
  278,
  2009,
  29889,
  13,
  13,
  2277,
  29937,
  2799,
  4080,
  29901,
  13,
  31999,
  30544,
  30287,
  31780,
  31144,
  30356,
  30626,
  30461,
  30214,
  31479,
  30287,
  30502,
  30783,
  30888,
  30698,
  233,
  154,
  136,
  233,
  187,
  187,
  31495,
  30940,
  30210,
  233,
  146,
  146,
  235,
  194,
  179,
  13,
  13,
  2277,
  29937,
  10567,
  29901,
  13,
  231,
  191,
  169,
  233,
  152,
  169,
  13,
  13,
  2277,
  29937,
  13291,
  29901,
  13,
  231,
  191,
  169,
  233,
  152,
  169,
  30878,
  235,
  148,
  154,
  30548,
  30210,
  233,
  154,
  136,
  233,
  187,
  187,
  31495,
  30940,
  30577,


In [19]:
data

Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

In [ ]:
import torch
import sentencepiece as spm
from transformers import GPTNeoForCausalLM, 
# tokenizer = spm.SentencePieceProcessor(model_file="mengzi_gpt.model")
model = GPTNeoForCausalLM.from_pretrained("Langboat/mengzi-gpt-neo-base")

def lm(prompt, top_k, top_p, max_length, repetition_penalty):
    input_ids = torch.tensor(tokenizer.encode([prompt]), dtype=torch.long, device='cuda')
    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        max_length=max_length+len(prompt),
        repetition_penalty=repetition_penalty)
    result = tokenizer.decode(gen_tokens.tolist())[0]
    return result


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("Langboat/mengzi-gpt-neo-base")